In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train=pd.read_csv(train_file_path,sep='\t',names=['Spam','message'])
test=pd.read_csv(test_file_path,sep='\t',names=['Spam','message'])
df=pd.concat([train,test])
df.reset_index(drop=True,inplace=True)

In [ ]:
df.head()

,Spam,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5571 entries, 0 to 5570
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Spam     5571 non-null   object
 1   message  5571 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
words=df.message[0]
print(words)

ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.


In [ ]:
def preprocessing(message):
    words=re.sub(r'[^a-zA-Z\s]','',message)
    words=word_tokenize(words)

    stop_words=set(stopwords.words('english'))

    words=[W for W in words if W not in stop_words]

    stemmer=PorterStemmer()
    words=[stemmer.stem(word) for word in words]

    return' '.join(words)

In [ ]:
df['message']=df['message'].apply(preprocessing)
df['Spam'].replace(['ham','spam'],[0,1],inplace=True)
df.head()

,Spam,message
0,0,ahhhhjust woken uphad bad dream u thoso dont l...
1,0,never noth
2,0,u sound like manki scous boy stevelik travel d...
3,0,mum say wan go go shun bian watch da glass exh...
4,0,never lei v lazi got wat dat day send da url c...


In [ ]:
df['Spam'].value_counts()

0    4824
1     747
Name: Spam, dtype: int64

In [ ]:
#Creating sub-sumple with reduced class imbalance
df=df.sample(frac=1)
spam=df[df['Spam']==1]
ham=df[df['Spam']==0][:1500]
newDf=pd.concat([spam,ham]).sample(frac=1,random_state=5)
newDf.head()

,Spam,message
528,0,met stranger choos friend long world stand fri...
5413,0,fine miss much
1464,0,merri christma babe love ya kiss
5380,0,thank yesterday sir wonder hope enjoy burial m...
5082,1,hot n horni will live local text repli hear st...


In [ ]:
X=newDf['message']
y=newDf['Spam']
tfidf=TfidfVectorizer(max_features=5000)
X=tfidf.fit_transform(X)

In [ ]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=28)

In [ ]:
clf=MultinomialNB()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98       299
           1       1.00      0.91      0.95       151

    accuracy                           0.97       450
   macro avg       0.98      0.95      0.96       450
weighted avg       0.97      0.97      0.97       450



In [ ]:
print(X.shape)

(2247, 4373)


In [ ]:
# New sentence to test
new_sentence = "Congratulations! You've won a free vacation. Click the link to claim your prize."
new_sentence=preprocessing(new_sentence)
# Vectorize the new sentence using the same TF-IDF vectorizer
new_sentence_tfidf = tfidf.transform([new_sentence])

# Make a prediction for the new sentence
prediction = clf.predict_proba(new_sentence_tfidf)

# Check the prediction result
print(prediction)

[[0.03657248 0.96342752]]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred_text=preprocessing(pred_text)
  pred_text=tfidf.transform([pred_text])
  pred = clf.predict_proba(pred_text)
  if pred[0][1]>=0.5:
     prediction=[pred[0][1],'spam']
  else:
     prediction=[pred[0][0],'ham']

  return (prediction)

pred_text = "our new mobile video service is live. just install on your phone to start watching."

prediction = predict_message(pred_text)
print(prediction)

[0.7479951708431385, 'spam']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]


  results=[]
  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    results.append(prediction[1])
    if prediction[1] != ans:
      passed = False
  print(results)
  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


['ham', 'spam', 'ham', 'spam', 'spam', 'ham', 'ham']
You passed the challenge. Great job!
